In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import dct  # Ensure this import for DCT
from PIL import Image
from gammatone.gtgram import gtgram

def compute_gtcc(log_gtgram, num_ceps):
   
    # Compute DCT along the frequency axis
    gtcc = dct(log_gtgram, type=2, axis=1, norm='ortho')[:, 1:num_ceps+1]
    return gtcc

def resize_and_save_image(gtcc_features, output_directory, filename, size=(224, 224)):
    
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)
    
    # Display the GTCC
    plt.imshow(gtcc_features, aspect='auto', origin='lower')
    plt.axis('off')  # Remove axis for a cleaner image

    # Save the figure to a temporary file
    temp_filepath = "temp_image.png"
    plt.savefig(temp_filepath, bbox_inches='tight', pad_inches=0.0, transparent=True)
    plt.close()
    
    # Resize the image using PIL
    try:
        image = Image.open(temp_filepath)
        resized_image = image.resize(size, Image.LANCZOS)
        
        # Save the resized image
        resized_image.save(os.path.join(output_directory, filename))
    except Exception as e:
        print(f"Error resizing and saving image: {e}")
    finally:
        # Optionally remove the temporary file
        if os.path.exists(temp_filepath):
            os.remove(temp_filepath)

# Base directory containing CSV files
base = r"D:\csv files"

# Iterate through folders in the base directory
for folder in os.listdir(base):
    file_path = os.path.join(base, folder)
    snr = [6.0,-6.0]
    
    for j in snr:
        path = os.path.join(file_path, f"{j}.csv")
        
        # Load data from CSV file, skipping the header
        data = np.genfromtxt(path, delimiter=",", skip_header=1)
        
        # Determine the number of subframes based on the folder name
        if folder in ["WOLA_QAM16", "WOLA_QAM64", "FOFDM_QAM16", "FOFDM_QAM64"]:
            num_subframes = 3627
        else:
            num_subframes = 3626
        
        # Reshape data
        replicas = data.reshape((1000, num_subframes))
        
        # Parameters for gammatone filterbank
        sample_rate = 44100
        f_min = 50  # Minimum frequency
        f_max = sample_rate / 2  # Maximum frequency
        num_filters = 32  # Number of filters in the gammatone filterbank
        window_time = 0.025  # Window length in seconds
        hop_time = 0.010  # Hop length in seconds

        # Define dataset types
        typ = ['training', 'testing', 'validation']
        
        for i in typ:
            output_directory = os.path.join("D:/spectogram_gtcc", str(j), i, folder)
            
            for replica in range(50):
                # Extract the signal from the row
                signal = replicas[replica]
                
                # Normalize the signal
                signal = signal / np.max(np.abs(signal))
                
                # Compute the gammatone spectrogram
                gtgram_matrix = gtgram(signal, sample_rate, window_time, hop_time, num_filters, f_min)
                
                # Logarithmic compression
                log_gtgram = np.log1p(gtgram_matrix)
                
                # Compute GTCC
                num_ceps = 13  # Number of cepstral coefficients (adjust as needed)
                gtcc_features = compute_gtcc(log_gtgram, num_ceps)
                if gtcc_features.ndim == 1:
                    gtcc_features = gtcc_features.reshape((num_ceps, -1))
                
                # Save the resized GTCC image
                filename = f"replica_{replica}.png"
                resize_and_save_image(gtcc_features, output_directory, filename, size=(224, 224))

